In [1]:
from pathlib import Path
import shutil
import json
import random

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import box
from shapely.affinity import translate, scale
from shapely.strtree import STRtree
from shapely.ops import nearest_points, unary_union

from pyrosm import OSM
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from sklearn.neighbors import NearestNeighbors

In [2]:
# -------------------- CONFIG --------------------
plt.rcParams["figure.dpi"] = 300

PBF_PATH = "../data/input/koeln-regbez-250927.osm.pbf"
BBOX = [7.00, 50.65, 7.20, 50.82]      # (min_lon, min_lat, max_lon, max_lat) around Bonn
TARGET_CRS = "EPSG:25832"              # metric CRS (UTM 32N)
TILE_SIZE_M = 400                      # tile width/height in meters
TOP_K = 824                            # how many tiles to render

FIG_INCH = 6
DPI = 300
DRAW_TILE_FRAME = True

IMPORTANT_ROADS = {
    "motorway", "trunk", "primary", "secondary", "tertiary",
    "motorway_link", "trunk_link", "primary_link", "secondary_link", "tertiary_link",
    "residential", "unclassified",
}
WIDTH_MAP = {
    "motorway": 2.0, "trunk": 1.8, "primary": 1.6,
    "secondary": 1.4, "tertiary": 1.2,
    "residential": 0.8, "unclassified": 0.8,
    "motorway_link": 1.4, "trunk_link": 1.3,
    "primary_link": 1.2, "secondary_link": 1.1, "tertiary_link": 1.0,
}
ROAD_COLOR = "black"
BLDG_FACE = "gray"
BLDG_EDGE = "white"

SEED = 42

OUT_DIR = Path("../data/input/samples/metadata_new")
SAMPLES_DIR = Path("../data/input/samples/pairs_new")
SUMMARY_DIR = Path("../data/input/samples/samples_summary_new")

RENDER_PNG = True  # (kept for future; currently always rendering)

In [3]:
# -------------------- GENERIC HELPERS --------------------
def make_grid(gdf: gpd.GeoDataFrame, size: float) -> gpd.GeoDataFrame:
    xmin, ymin, xmax, ymax = gdf.total_bounds
    xs = np.arange(xmin, xmax, size)
    ys = np.arange(ymin, ymax, size)
    polys = [box(x, y, x + size, y + size) for x in xs for y in ys]
    return gpd.GeoDataFrame({"tile_id": range(len(polys))}, geometry=polys, crs=gdf.crs)


def compute_tolerance(tile_size_m: float, fig_in: float, dpi: int) -> float:
    px = fig_in * dpi
    mpp = tile_size_m / px
    return mpp * 0.75  # safe simplification to pixel scale


def band(val, pct=0.20, lo=None, hi=None):
    """Make a [lo, hi] band around a center value with % width and clamps."""
    a = float(val) * (1 - pct)
    b = float(val) * (1 + pct)
    if lo is not None:
        a = max(lo, a)
    if hi is not None:
        b = min(hi, b)
    if b <= a:
        b = a + ((hi - lo) * 0.05 if (hi is not None and lo is not None) else 1e-6)
    return (round(a, 3), round(b, 3))


def balanced_split(ids, groups):
    """Split list `ids` into len(groups) parts, sizes differ by at most 1."""
    n = len(ids)
    g = len(groups)
    base = n // g
    rem = n % g
    parts = []
    start = 0
    for i in range(g):
        size = base + (1 if i < rem else 0)
        parts.append(ids[start : start + size])
        start += size
    return dict(zip(groups, parts))


def _normalize_gdf(gdf, xmin, ymin, sx, sy):
    """Shift & scale a GeoDataFrame into normalized 0..400 space."""
    if gdf is None or gdf.empty:
        return gdf
    gdf = gdf.copy()
    gdf["geometry"] = gdf["geometry"].apply(
        lambda geom: scale(
            translate(geom, xoff=-xmin, yoff=-ymin),  # move tile origin to (0,0)
            xfact=sx,
            yfact=sy,
            origin=(0, 0),
        )
    )
    return gdf

In [4]:
# -------------------- GENERALIZATION OPERATORS --------------------
def aggregate_buildings(gdf, dist, join_style=2, mitre_limit=5.0, cap_style=2, resolution=1):
    """
    Buffer(+dist) → dissolve → buffer(-dist) with straight (mitre) corners.
    """
    if gdf.empty:
        return gdf.copy()

    buff = gdf.geometry.buffer(
        dist,
        join_style=join_style,
        mitre_limit=mitre_limit,
        cap_style=cap_style,
        resolution=resolution,
    )
    merged = unary_union(buff)
    out = gpd.GeoDataFrame(geometry=[merged], crs=gdf.crs)
    out["geometry"] = out.geometry.buffer(
        -dist,
        join_style=join_style,
        mitre_limit=mitre_limit,
        cap_style=cap_style,
        resolution=resolution,
    )
    out = out[~out.geometry.is_empty & out.geometry.is_valid]
    return out.explode(index_parts=False).reset_index(drop=True)


def simplify_buildings(gdf, eps):
    """Douglas–Peucker simplification on polygon boundaries."""
    if gdf.empty:
        return gdf.copy()
    out = gdf.copy()
    out["geometry"] = out.geometry.simplify(eps, preserve_topology=True)
    out = out[~out.geometry.is_empty & out.geometry.is_valid]
    return out


def displace_buildings(
    gdf,
    clearance,
    iters=40,
    step=1.2,
    max_total=10.0,
    small_moves_more=True,
    area_ref=120.0,
):
    """
    Edge-aware repulsion:
      • neighbors via STRtree within 'clearance'
      • push along nearest-edge direction
      • small buildings move more than large ones
    """
    if gdf.empty:
        return gdf.copy()

    out = gdf.copy()
    geoms = list(out.geometry.values)
    areas = np.maximum(1.0, np.array([g.area for g in geoms]))

    if small_moves_more:
        weights = np.clip((area_ref / areas), 0.5, 3.0)
    else:
        weights = np.ones_like(areas)

    tree = STRtree(geoms)
    offsets = np.zeros((len(geoms), 2), dtype=float)

    for _ in range(iters):
        moved = np.zeros_like(offsets)
        any_push = False

        for i, gi in enumerate(geoms):
            cand_idx = [
                j for j in tree.query(gi.buffer(clearance).envelope) if j != i
            ]
            if not cand_idx:
                continue

            vi = np.array([0.0, 0.0])
            for j in cand_idx:
                gj = geoms[j]
                d = gi.distance(gj)
                if d >= clearance or d == 0:
                    continue

                pi, pj = nearest_points(gi, gj)
                dir_vec = np.array([pi.x - pj.x, pi.y - pj.y])
                nrm = np.linalg.norm(dir_vec)
                if nrm == 0:
                    continue
                dir_vec /= nrm

                deficit = clearance - d
                push_i = 0.5 * deficit * (weights[i] / (weights[i] + weights[j] + 1e-6))
                vi += dir_vec * push_i
                any_push = True

            nrm = np.linalg.norm(vi)
            if nrm > 0:
                vi = (vi / nrm) * min(step, nrm)
            moved[i] = vi

        offsets += moved
        norms = np.linalg.norm(offsets, axis=1)
        too_far = norms > max_total
        if np.any(too_far):
            scale_f = (max_total / (norms[too_far] + 1e-6)).reshape(-1, 1)
            offsets[too_far] *= scale_f

        geoms = [
            translate(g, xoff=float(dx), yoff=float(dy))
            for g, (dx, dy) in zip(geoms, moved)
        ]

        if not any_push or np.max(np.linalg.norm(moved, axis=1)) < 1e-3:
            break

    out["geometry"] = geoms
    out = out[~out.geometry.is_empty & out.geometry.is_valid]
    return out


def select_buildings(gdf, area_threshold):
    """Remove polygons smaller than area_threshold (m²)."""
    if gdf.empty:
        return gdf.copy()
    out = gdf.copy()
    areas = out.geometry.area
    return out[areas >= area_threshold]

In [5]:
# -------------------- RENDERING --------------------
def render_single_frame(tid, gdf_buildings, tile_poly, roads_in_tiles, out_path):
    """
    Render one map frame with:
      - map normalized to 0..400 in both directions
      - axes labelled in meters (0 m .. 400 m)
      - light grid lines near axes (no features below 0).
    """
    fig, ax = plt.subplots(figsize=(FIG_INCH, FIG_INCH))

    # tile extent in original coordinates
    xmin, ymin, xmax, ymax = tile_poly.bounds
    width = xmax - xmin
    height = ymax - ymin

    TARGET = 400.0
    sx = TARGET / width
    sy = TARGET / height

    # subset & normalize roads and buildings
    r_tile = roads_in_tiles[roads_in_tiles.tile_id == tid]
    r_tile_n = _normalize_gdf(r_tile, xmin, ymin, sx, sy)
    bldg_n = _normalize_gdf(gdf_buildings, xmin, ymin, sx, sy)

    # roads
    if r_tile_n is not None and not r_tile_n.empty:
        if "highway" in r_tile_n.columns:
            for cls, df in r_tile_n.groupby("highway"):
                lw = WIDTH_MAP.get(cls, 0.8)
                df.plot(ax=ax, color=ROAD_COLOR, linewidth=lw, zorder=2)
        else:
            r_tile_n.plot(ax=ax, color=ROAD_COLOR, linewidth=0.9, zorder=2)

    # buildings
    if bldg_n is not None and not bldg_n.empty:
        bldg_n.plot(
            ax=ax,
            facecolor=BLDG_FACE,
            edgecolor=BLDG_EDGE,
            linewidth=0.3,
            zorder=3,
        )

    # normalized axes
    ax.set_xlim(0, TARGET)
    ax.set_ylim(0, TARGET)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    nticks = 5
    ticks = np.linspace(0, TARGET, nticks)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)

    # tick labels with "m"
    meter_formatter = FuncFormatter(lambda v, pos: f"{int(v)} m")
    ax.xaxis.set_major_formatter(meter_formatter)
    ax.yaxis.set_major_formatter(meter_formatter)

    # light grid close to axes
    grid_height = TARGET * 0.05
    for x in ticks:
        ax.plot([x, x], [0, grid_height], color="0.85", lw=0.5, zorder=1)

    grid_width = TARGET * 0.05
    for y in ticks:
        ax.plot([0, grid_width], [y, y], color="0.85", lw=0.5, zorder=1)

    ax.set_xlabel("")
    ax.set_ylabel("")

    plt.savefig(out_path, dpi=DPI, bbox_inches="tight", pad_inches=0.05)
    plt.close(fig)


In [6]:
# -------------------- LOAD OSM & BASIC PREP --------------------
osm = OSM(PBF_PATH, bounding_box=BBOX)
buildings = osm.get_buildings()
roads = osm.get_network(network_type="all")  # try "driving" for fewer lines

print(f"Raw buildings: {0 if buildings is None else len(buildings)}")
print(f"Raw roads    : {0 if roads is None else len(roads)}")

if buildings is None or len(buildings) == 0:
    raise RuntimeError("No buildings found in this bbox.")

buildings = buildings.to_crs(TARGET_CRS)
if roads is not None and len(roads) > 0:
    roads = roads.to_crs(TARGET_CRS)

if roads is not None and len(roads) > 0 and "highway" in roads.columns:
    roads = roads[roads["highway"].isin(IMPORTANT_ROADS)].copy()

print("CRS:", buildings.crs, (None if roads is None else roads.crs))

# grid & tile selection
grid = make_grid(buildings, size=TILE_SIZE_M)

b_cent = buildings.copy()
b_cent["geometry"] = b_cent.geometry.centroid
join = gpd.sjoin(b_cent, grid[["tile_id", "geometry"]], how="left", predicate="within")
buildings["tile_id"] = join["tile_id"].values
buildings = buildings.dropna(subset=["tile_id"]).copy()
buildings["tile_id"] = buildings["tile_id"].astype(int)

counts = buildings.groupby("tile_id").size().rename("n_bldg")
top_ids = counts.nlargest(min(TOP_K, len(counts))).index

buildings_top = buildings[buildings["tile_id"].isin(top_ids)].copy()
grid_top = grid[grid["tile_id"].isin(top_ids)].copy()

# roads per tile
roads_in_tiles = gpd.GeoDataFrame(
    columns=["geometry", "highway", "tile_id"], crs=TARGET_CRS
)
if roads is not None and len(roads) > 0:
    grid_pad = grid_top.copy()
    grid_pad["geometry"] = grid_pad.geometry.buffer(0.5)  # catch edge lines
    roads_in_tiles = gpd.sjoin(
        roads[["geometry", "highway"]] if "highway" in roads.columns else roads[["geometry"]],
        grid_pad[["tile_id", "geometry"]],
        how="inner",
        predicate="intersects",
    ).drop(columns="index_right")
    if "highway" not in roads_in_tiles.columns:
        roads_in_tiles["highway"] = None

# simplify to pixel scale
tol = compute_tolerance(TILE_SIZE_M, FIG_INCH, DPI)
buildings_top = buildings_top.set_geometry(
    buildings_top.geometry.simplify(tol, preserve_topology=True)
)
if len(roads_in_tiles) > 0:
    roads_in_tiles = roads_in_tiles.set_geometry(
        roads_in_tiles.geometry.simplify(tol * 1.2, preserve_topology=True)
    )

/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/constructive.py:246: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/constructive.py:246: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/pyrosm/networks.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a singl

Raw buildings: 188628
Raw roads    : 64576
CRS: EPSG:25832 EPSG:25832


/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/constructive.py:1177: RuntimeWarning: invalid value encountered in simplify_preserve_topology
  return lib.simplify_preserve_topology(geometry, tolerance, **kwargs)


In [7]:
# -------------------- PARAMETER BANDS --------------------
np.random.seed(SEED)
random.seed(SEED)

OUT_DIR.mkdir(parents=True, exist_ok=True)
if SAMPLES_DIR.exists():
    shutil.rmtree(SAMPLES_DIR)
SAMPLES_DIR.mkdir(parents=True, exist_ok=True)

g = buildings_top[["geometry"]].copy()
assert len(g) > 0, "No buildings in buildings_top."

areas = g.area
if len(areas) >= 5:
    A_low_q, A_med_q, A_high_q = np.quantile(areas, [0.10, 0.25, 0.40])
else:
    A_low_q, A_med_q, A_high_q = 40.0, 80.0, 140.0

cent = np.c_[
    g.geometry.centroid.x.values,
    g.geometry.centroid.y.values,
]
if len(cent) >= 2:
    nbrs = NearestNeighbors(n_neighbors=2).fit(cent)
    dists = np.sort(nbrs.kneighbors(cent, return_distance=True)[0][:, 1])
    D_low, D_med, D_high = np.quantile(dists, [0.25, 0.50, 0.75])
else:
    D_low, D_med, D_high = 2.0, 4.0, 7.0

print("Area quantiles (m²)  ~ 10/25/40%:", round(A_low_q, 1), round(A_med_q, 1), round(A_high_q, 1))
print("NN distances (m)     ~ 25/50/75%:", round(D_low, 2), round(D_med, 2), round(D_high, 2))

# aggregation
agg_low_c  = np.clip(max(2.0, D_low * 0.6),  2.0, 12.0)
agg_med_c  = np.clip(max(3.0, D_med * 1.0),  3.0, 12.0)
agg_high_c = np.clip(D_high * 1.5,          4.0, 12.0)

# displacement
disp_low_c  = np.clip(max(0.6, D_low * 0.3), 0.6, 3.0)
disp_med_c  = np.clip(max(1.2, D_med * 0.4), 1.0, 3.0)
disp_high_c = np.clip(D_high * 0.6,          1.5, 3.0)

# simplification
simp_low_c  = 0.5
simp_med_c  = min(3.0, max(1.0, D_low * 0.8))
simp_high_c = min(7.0, max(3.0, D_med * 1.0))

# selection shift
SHIFT_MODE = "add"   # "add" | "mul" | "pctile"
SEL_ADD = {"low": 20.0, "medium": 50.0, "high": 90.0}
SEL_MUL = {"low": 1.25, "medium": 1.50, "high": 1.80}
ALT_Q = {"low": 0.20, "medium": 0.35, "high": 0.55}
SEL_MIN = {"low": 30.0, "medium": 70.0,  "high": 120.0}
SEL_MAX = {"low": 220.0, "medium": 260.0, "high": 320.0}


def shifted_selection_centers():
    if SHIFT_MODE == "add":
        c_low  = A_low_q  + SEL_ADD["low"]
        c_med  = A_med_q  + SEL_ADD["medium"]
        c_high = A_high_q + SEL_ADD["high"]
    elif SHIFT_MODE == "mul":
        c_low  = A_low_q  * SEL_MUL["low"]
        c_med  = A_med_q  * SEL_MUL["medium"]
        c_high = A_high_q * SEL_MUL["high"]
    elif SHIFT_MODE == "pctile":
        q_low, q_med, q_high = np.quantile(areas, [ALT_Q["low"], ALT_Q["medium"], ALT_Q["high"]])
        c_low, c_med, c_high = q_low, q_med, q_high
    else:
        raise ValueError("SHIFT_MODE must be 'add', 'mul', or 'pctile'")
    c_low  = float(np.clip(c_low,  SEL_MIN["low"],    SEL_MAX["low"]))
    c_med  = float(np.clip(c_med,  SEL_MIN["medium"], SEL_MAX["medium"]))
    c_high = float(np.clip(c_high, SEL_MIN["high"],   SEL_MAX["high"]))
    return c_low, c_med, c_high


sel_low_c, sel_med_c, sel_high_c = shifted_selection_centers()

PARAM_SCHEMES = {
    "aggregate": {
        "low":    band(agg_low_c,  pct=0.20, lo=2.0,  hi=12.0),
        "medium": band(agg_med_c,  pct=0.20, lo=3.0,  hi=12.0),
        "high":   band(agg_high_c, pct=0.20, lo=4.0,  hi=12.0),
        "unit": "m",
    },
    "simplify": {
        "low":    band(simp_low_c,  pct=0.20, lo=0.3, hi=2.0),
        "medium": band(simp_med_c,  pct=0.20, lo=1.0, hi=4.0),
        "high":   band(simp_high_c, pct=0.20, lo=3.0, hi=7.0),
        "unit": "m",
    },
    "displace": {
        "low":    band(disp_low_c,  pct=0.20, lo=0.5, hi=1.2),
        "medium": band(disp_med_c,  pct=0.20, lo=1.0, hi=2.0),
        "high":   band(disp_high_c, pct=0.20, lo=1.6, hi=3.0),
        "unit": "m",
    },
    "select": {
        "low":    band(sel_low_c,  pct=0.20, lo=SEL_MIN["low"],    hi=SEL_MAX["low"]),
        "medium": band(sel_med_c,  pct=0.20, lo=SEL_MIN["medium"], hi=SEL_MAX["medium"]),
        "high":   band(sel_high_c, pct=0.20, lo=SEL_MIN["high"],   hi=SEL_MAX["high"]),
        "unit": "m^2",
    },
}

print("\nAuto-tuned parameter bands:")
for op in ["aggregate", "simplify", "displace", "select"]:
    print(
        f"{op:10s}  low={PARAM_SCHEMES[op]['low']}  "
        f"med={PARAM_SCHEMES[op]['medium']}  high={PARAM_SCHEMES[op]['high']}  "
        f"({PARAM_SCHEMES[op]['unit']})"
    )

OPERATORS = list(PARAM_SCHEMES.keys())
INTENSITIES = ["low", "medium", "high"]


def sample_param(op, intensity):
    lo, hi = PARAM_SCHEMES[op][intensity][:2]
    return float(np.random.uniform(lo, hi))

Area quantiles (m²)  ~ 10/25/40%: 17.4 28.2 60.9
NN distances (m)     ~ 25/50/75%: 5.75 7.79 12.12

Auto-tuned parameter bands:
aggregate   low=(2.759, 4.139)  med=(6.229, 9.343)  high=(9.6, 12.0)  (m)
simplify    low=(0.4, 0.6)  med=(2.4, 3.6)  high=(5.6, 7.0)  (m)
displace    low=(1.38, 1.415)  med=(2.4, 2.45)  high=(2.4, 3.0)  (m)
select      low=(30.0, 44.897)  med=(70.0, 93.883)  high=(120.707, 181.061)  (m^2)


In [8]:
# -------------------- ASSIGNMENT OF TILES → (OP, INTENSITY) --------------------
assert "tile_id" in buildings_top.columns
assert "tile_id" in grid_top.columns

tile_ids = sorted(buildings_top["tile_id"].unique().tolist())
print(f"Tiles available: {len(tile_ids)}")

ids_shuffled = tile_ids.copy()
random.shuffle(ids_shuffled)

op_assign = balanced_split(ids_shuffled, OPERATORS)

assignments = []
for op, ids_for_op in op_assign.items():
    random.shuffle(ids_for_op)
    split_int = balanced_split(ids_for_op, INTENSITIES)
    for intensity, ids_for_int in split_int.items():
        for tid in ids_for_int:
            assignments.append(
                {
                    "tile_id": tid,
                    "operator": op,
                    "intensity": intensity,
                    "param_value": sample_param(op, intensity),
                    "param_unit": PARAM_SCHEMES[op]["unit"],
                }
            )

assign_df = (
    pd.DataFrame(assignments)
    .sort_values(["operator", "intensity", "tile_id"])
    .reset_index(drop=True)
)


Tiles available: 824


In [9]:
# -------------------- MAIN SAMPLE GENERATION LOOP --------------------
meta_rows = []

for _, row in assign_df.iterrows():
    tid = int(row.tile_id)
    op = row.operator
    inten = row.intensity
    param_val = float(row.param_value)
    unit = row.param_unit

    sample_dir = SAMPLES_DIR / f"{tid:04d}"
    sample_dir.mkdir(parents=True, exist_ok=True)

    g_tile = buildings_top.loc[buildings_top.tile_id == tid][["geometry"]].copy()
    tile_poly = grid_top.loc[grid_top.tile_id == tid].geometry.iloc[0]

    # INPUT GeoJSON
    in_geo_path = sample_dir / f"{tid:04d}_input.geojson"
    if not in_geo_path.exists():
        g_tile.to_file(in_geo_path, driver="GeoJSON")

    # INPUT PNG
    in_png_path = sample_dir / f"input_{tid:04d}.png"
    render_single_frame(tid, g_tile, tile_poly, roads_in_tiles, in_png_path)

    # apply operator
    if op == "aggregate":
        out_gdf = aggregate_buildings(g_tile, dist=param_val)
    elif op == "simplify":
        out_gdf = simplify_buildings(g_tile, eps=param_val)
    elif op == "displace":
        out_gdf = displace_buildings(g_tile, clearance=param_val, iters=15, step=0.6)
    elif op == "select":
        out_gdf = select_buildings(g_tile, area_threshold=param_val)
    else:
        raise ValueError(op)

    # TARGET GeoJSON
    out_geo_path = sample_dir / f"{tid:04d}_generalized.geojson"
    out_gdf.to_file(out_geo_path, driver="GeoJSON")

    # TARGET PNG
    out_png_path = sample_dir / f"generalized_{tid:04d}.png"
    render_single_frame(tid, out_gdf, tile_poly, roads_in_tiles, out_png_path)

    meta_rows.append(
        {
            "sample_id": tid,
            "operator": op,
            "intensity": inten,
            "param_value": round(param_val, 3),
            "param_unit": unit,
            "input_geojson": str(in_geo_path),
            "target_geojson": str(out_geo_path),
            "input_png": str(in_png_path),
            "target_png": str(out_png_path),
            "prompt": "",
            "notes": "",
            "n_input_polys": int(g_tile.shape[0]),
            "n_target_polys": int(out_gdf.shape[0]),
            "is_target_empty": bool(out_gdf.empty),
        }
    )

meta_df = pd.DataFrame(meta_rows)
meta_path = OUT_DIR / "meta.csv"
meta_df.to_csv(meta_path, index=False)
print("Saved:", meta_path)

print("\nCounts by operator:")
display(meta_df["operator"].value_counts())

print("\nOperator × intensity:")
display(pd.crosstab(meta_df["operator"], meta_df["intensity"]))

print("\nEmpty targets (should be few or none):", meta_df["is_target_empty"].sum())
display(meta_df.head(10))

/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/constructive.py:246: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/set_operations.py:553: RuntimeWarning: invalid value encountered in unary_union
  return lib.unary_union(collections, **kwargs)
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/constructive.py:246: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/constructive.py:246: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/set_operations.py:553: RuntimeWarning: invalid value encountered in unary_union
  return lib.unary_union(collections, **kwargs)
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/constructive.py:246: RuntimeWarning: invalid value encountered in buffer
  ret

Saved: ../data/input/samples/metadata_new/meta.csv

Counts by operator:


/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/shapely/constructive.py:1177: RuntimeWarning: invalid value encountered in simplify_preserve_topology
  return lib.simplify_preserve_topology(geometry, tolerance, **kwargs)


operator
aggregate    206
displace     206
select       206
simplify     206
Name: count, dtype: int64


Operator × intensity:


intensity,high,low,medium
operator,,,
aggregate,68,69,69
displace,68,69,69
select,68,69,69
simplify,68,69,69



Empty targets (should be few or none): 0


,sample_id,operator,intensity,param_value,param_unit,input_geojson,target_geojson,input_png,target_png,prompt,notes,n_input_polys,n_target_polys,is_target_empty
0,73,aggregate,high,11.753,m,../data/input/samples/pairs_new/0073/0073_inpu...,../data/input/samples/pairs_new/0073/0073_gene...,../data/input/samples/pairs_new/0073/input_007...,../data/input/samples/pairs_new/0073/generaliz...,,,381,4,False
1,78,aggregate,high,11.063,m,../data/input/samples/pairs_new/0078/0078_inpu...,../data/input/samples/pairs_new/0078/0078_gene...,../data/input/samples/pairs_new/0078/input_007...,../data/input/samples/pairs_new/0078/generaliz...,,,208,17,False
2,126,aggregate,high,11.148,m,../data/input/samples/pairs_new/0126/0126_inpu...,../data/input/samples/pairs_new/0126/0126_gene...,../data/input/samples/pairs_new/0126/input_012...,../data/input/samples/pairs_new/0126/generaliz...,,,119,8,False
3,141,aggregate,high,10.473,m,../data/input/samples/pairs_new/0141/0141_inpu...,../data/input/samples/pairs_new/0141/0141_gene...,../data/input/samples/pairs_new/0141/input_014...,../data/input/samples/pairs_new/0141/generaliz...,,,239,13,False
4,212,aggregate,high,10.284,m,../data/input/samples/pairs_new/0212/0212_inpu...,../data/input/samples/pairs_new/0212/0212_gene...,../data/input/samples/pairs_new/0212/input_021...,../data/input/samples/pairs_new/0212/generaliz...,,,87,17,False
5,215,aggregate,high,11.258,m,../data/input/samples/pairs_new/0215/0215_inpu...,../data/input/samples/pairs_new/0215/0215_gene...,../data/input/samples/pairs_new/0215/input_021...,../data/input/samples/pairs_new/0215/generaliz...,,,272,6,False
6,258,aggregate,high,11.428,m,../data/input/samples/pairs_new/0258/0258_inpu...,../data/input/samples/pairs_new/0258/0258_gene...,../data/input/samples/pairs_new/0258/input_025...,../data/input/samples/pairs_new/0258/generaliz...,,,131,16,False
7,286,aggregate,high,10.322,m,../data/input/samples/pairs_new/0286/0286_inpu...,../data/input/samples/pairs_new/0286/0286_gene...,../data/input/samples/pairs_new/0286/input_028...,../data/input/samples/pairs_new/0286/generaliz...,,,326,19,False
8,361,aggregate,high,10.829,m,../data/input/samples/pairs_new/0361/0361_inpu...,../data/input/samples/pairs_new/0361/0361_gene...,../data/input/samples/pairs_new/0361/input_036...,../data/input/samples/pairs_new/0361/generaliz...,,,163,16,False
9,390,aggregate,high,9.622,m,../data/input/samples/pairs_new/0390/0390_inpu...,../data/input/samples/pairs_new/0390/0390_gene...,../data/input/samples/pairs_new/0390/input_039...,../data/input/samples/pairs_new/0390/generaliz...,,,441,15,False
